In [14]:
#import module

import pandas
import pycaret
import numpy as np
import math
import matplotlib.pyplot as plt
from pycaret.regression import *
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

from pycaret.regression import load_model

import plotly.io as pio
pio.renderers.default = 'iframe_connected'

import time

In [ ]:
# import data

filename = "./Data_2021_10_14_v1 (N98923)/Data.csv"

raw_data = load_data(filename=fn, pp=1)